# Toolkit

In [1053]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Dataset

In [1054]:
iris = pd.read_csv("/content/Iris.csv")
iris.drop(columns = ['Id'], inplace = True)
iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


# Perceptron

## helper classes

In [1055]:
class Perceptron:
    def __init__(self, n_inputs):
        self.weights = np.random.randn(1, n_inputs)
        self.bias = np.random.randn(1, 1)
    

    def fit(self, X, y, learning_rate = 0.1, epochs = 50):
        for epoch in range(epochs):
            missclassification = 0

            for i, data in enumerate(zip(X, y)):
                inputs, targets = data

                # forward pass
                z = inputs @ self.weights.T + self.bias
                predictions = np.where(z >= 0, 1, -1)


                # loss calculation
                losses = targets - predictions
                missclassification += abs(losses).sum() / 2


                # backward pass
                self.weights = self.weights + learning_rate * (losses * inputs).mean(axis = 0)
                self.bias = self.bias + learning_rate * losses.mean(axis = 0)
            

            print('>>epoch=%d, missclassification=%.3f' % (epoch, missclassification))
        return self.weights, self.bias
    

    def predict(self, X):
        z = X @ self.weights.T + self.bias
        prediction = np.where(z >= 0, 1, -1)
        return prediction
    

    def accuracy(self, targets, preds):
        missclassifications = abs((preds.reshape(targets.shape) - targets)).sum() / 2
        error = missclassifications / len(targets)
        return 1 - error

## example on iris dataset

In [1056]:
X = iris.drop(columns = ['Species']).to_numpy()

y = iris.loc[:, 'Species']
y = np.where(y == 'Iris-setosa', 1, -1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y)

y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

# std_scaler = StandardScaler()
# X_train = std_scaler.fit_transform(X_train)
# X_test = std_scaler.transform(X_test)


print(X.shape, y.shape)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(150, 4) (150,)
(105, 4) (45, 4) (105, 1) (45, 1)


In [1057]:
model = Perceptron(X_train.shape[1])
model.fit(X_train, y_train, epochs = 2)

pred = model.predict(X_test)
acc = model.accuracy(y_test, pred)
print('accuracy: ', acc)

# print(model.weights)

>>epoch=0, missclassification=5.000
>>epoch=1, missclassification=0.000
accuracy:  1.0


# MLP

## helper classes

### activation choices

In [1058]:
class Sigmoid:
    def __init__(self):
        pass
    

    def forward(self, p_num):
        sigmoid_val = 1 / (1 + np.exp(-p_num))
        
        return sigmoid_val

    
    def backward(self, p_num):
        fx = self.forward(p_num)
        derivative = fx * (1 - fx)

        return derivative

In [1059]:
class Tanh:
    def __init__(self):
        pass
    
    def forward(self, p_num):
        ez_plus = np.exp(p_num)
        ez_minus = np.exp(-p_num)
        tanh_val = (ez_plus - ez_minus) / (ez_plus + ez_minus)

        return tanh_val
    

    def backward(self, p_num):
        fx = self.forward(p_num)
        derivative = 1 - (fx**2)

        return derivative

In [1060]:
class ReLu:
    def __init__(self):
        pass
    
    def forward(self, p_num):
        relu_num = np.maximum(0, p_num)

        return relu_num
    
    
    def backward(self, p_num):
        derivative = (p_num > 0).astype(int)

        return derivative

### neural layer manager

In [1061]:
'''
# n_inputs = #neurons in previous layers (m) OR
#            #features in input (m)
# n_neurons = #neurons in this layer (n)

# weights.shape = (m, n) : n output nodes from layer l, m input nodes from layer l-1
# biases.shape = (1, n) : n output nodes

# inputs.shape = (k, m) : k examples, m features
# outputs.shape = (k, n) : k examples, n nodes at layer l
'''


# single layer management class
class Layer:
    def __init__(self, n_inputs, n_neurons, activation, weights_init = 'random', biases_init = 'random'):
        self.weights = None
        self.biases = None
        self.activation = activation
        
        self._reset_params(weights_init, biases_init, n_neurons, n_inputs)
    

    def forward(self, inputs):
        self.inputs = inputs
        self.z = self.inputs @ self.weights + self.biases
        self.a = self.activation.forward(self.z)

        return self.a


    def backward(self, delta, learning_rate, i):
        dL_dal = delta
        dal_dzl = np.diag(self.activation.backward(self.z)[0])
        dzl_dal_1 = self.weights.T
        dzl_dwl = self.inputs.T.reshape(self.inputs.T.shape[0], 1)
        dzl_dbl = np.ones(self.biases.shape)

        dL_dzl = dL_dal @ dal_dzl

        self.weights = self.weights - learning_rate * (dzl_dwl @ dL_dzl)
        self.biases = self.biases - learning_rate * (dL_dzl)
        
        return dL_dzl @ dzl_dal_1


    def _reset_params(self, weights_init, biases_init, n_neurons, n_inputs):
        weights_switcher = {
            'random' : self._random_weights,
            'zero' : self._zero_weights,
            'one' : self._one_weights
        }
        weights_fxn = weights_switcher.get(weights_init, 1)
        if weights_fxn == 1:
            print("INVALID WEIGHTS TYPE SELECTED!!")
            return weights_fxn;
            
        biases_switcher = {
            'random' : self._random_biases,
            'zero' : self._zero_biases,
            'one' : self._one_biases
        }
        biases_fxn = biases_switcher.get(biases_init, 2)
        if biases_fxn == 2:
            print("INVALID biases TYPE SELECTED!!")
            return biases_fxn;
        
        self.weights = weights_fxn(n_inputs, n_neurons)
        self.biases = biases_fxn(n_neurons)


    # weight init methods
    def _random_weights(self, n_inputs, n_neurons):
        weights = np.random.randn(n_inputs, n_neurons)
        return weights
        
    def _zero_weights(self, n_inputs, n_neurons):
        weights = np.zeros((n_inputs, n_neurons))
        return weights
        
    def _one_weights(self, n_inputs, n_neurons):
        weights = np.ones((n_inputs, n_neurons))
        return weights
    

    # bias init methods
    def _random_biases(self, n_neurons):
        biases = np.random.randn(1, n_neurons)
        return biases
        
    def _zero_biases(self, n_neurons):
        biases = np.zeros((1, n_neurons))
        return biases
        
    def _one_biases(self, n_neurons):
        biases = np.ones((1, n_neurons))
        return biases

### loss function choices

In [1062]:
class MSE:
    def __init__(self):
        pass
    
    def error(self, preds, targets):
        losses = (preds - targets)
        mse = 0.5 * (losses **2).mean()
        return mse
    
    def derivative(self, preds, targets):
        losses = preds - targets
        return losses

### MLP implementation

In [1063]:
class MLP:
    def __init__(self):
        self.layers = None
        pass
    
    def add_layer(self, n_inputs, n_neurons, activation, weights_init = 'random', biases_init = 'one'):
        if (self.layers == None):
            self.layers = []
        
        new_layer = Layer(n_inputs, n_neurons, activation, weights_init, biases_init)
        self.layers.append(new_layer)


    def train(self, X, y, loss_fxn, epochs = 50, n_batches = 1, learning_rate = 0.1):

        for epoch in range(epochs):
            data_batches = zip(X, y) #self._create_batches(X, y, n_batches)
            net_loss = 0.0
        
            for i, data_batch in enumerate(data_batches):
                inputs, targets = data_batch

                outputs = self._forward(inputs)

                loss = loss_fxn.error(outputs, targets)
                net_loss += loss

                d_loss = loss_fxn.derivative(outputs, targets)
                self._backward(d_loss, learning_rate)

            print('>>epoch=%d, net loss=%.3f' % (epoch, net_loss))


    def _forward(self, X):
        input = X
        for i, layer in enumerate(self.layers):
            output = layer.forward(input)
            input = output

        return input
    

    def _backward(self, d_loss, learning_rate):
        delta = d_loss #dL_da OR dL_dy
        for i, layer in enumerate(self.layers[::-1]):
            delta = layer.backward(delta, learning_rate, i)


    def predict(self, X):
        outputs = self._forward(X)
        preds = np.argmax(outputs, axis = 1)
        return preds


    def accuracy(self, y_pred, y_test):
        true = np.argmax(y_test, axis = 1)
        count = 0
        for i in range(len(true)):
            if (true[i] == y_pred[i]):
                count += 1
        acc = count / len(true)
        
        return acc


    def _create_batches(self, X, y, n_batches):
        mini_batches = []
        data = np.hstack((X, y))
        np.random.shuffle(data)
        
        batch_size = int(data.shape[0] / n_batches)
        i = 0
    
        for i in range(n_batches + 1):
            mini_batch = data[i * batch_size : (i + 1)*batch_size, :]
            X_mini = mini_batch[:, :-1]
            Y_mini = mini_batch[:, -1].reshape((-1, 1))
            mini_batches.append((X_mini, Y_mini))

        if data.shape[0] % batch_size != 0:
            mini_batch = data[i * batch_size:data.shape[0]]
            X_mini = mini_batch[:, :-1]
            Y_mini = mini_batch[:, -1].reshape((-1, 1))
            mini_batches.append((X_mini, Y_mini))
        return mini_batches

## example on iris dataset

In [1064]:
X = iris.drop(columns = ['Species']).to_numpy()

target_classes = np.array(iris.Species.unique())
target = iris.Species.to_numpy()
y = np.array([(i == target_classes).astype(dtype=int) for i in target])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y)

std_scaler = StandardScaler()
X_train = std_scaler.fit_transform(X_train)
X_test = std_scaler.transform(X_test)

print(X.shape, y.shape)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(150, 4) (150, 3)
(105, 4) (45, 4) (105, 3) (45, 3)


In [1065]:
model = MLP()

model.add_layer(X_train.shape[1], 4, Tanh(), biases_init = 'random')
model.add_layer(4, 5, Tanh(), biases_init = 'random')
model.add_layer(5, 3, Sigmoid(), biases_init = 'random')

model.train(X_train, y_train, MSE(), epochs = 15, learning_rate = 1)

>>epoch=0, net loss=6.376
>>epoch=1, net loss=3.172
>>epoch=2, net loss=3.376
>>epoch=3, net loss=2.911
>>epoch=4, net loss=1.378
>>epoch=5, net loss=1.422
>>epoch=6, net loss=1.837
>>epoch=7, net loss=1.542
>>epoch=8, net loss=2.121
>>epoch=9, net loss=1.757
>>epoch=10, net loss=1.720
>>epoch=11, net loss=1.278
>>epoch=12, net loss=1.207
>>epoch=13, net loss=1.518
>>epoch=14, net loss=1.235


In [1066]:
y_pred = model.predict(X_test)
acc = model.accuracy(y_pred, y_test)
print("accuracy: ", acc)

accuracy:  1.0
